# Generación de datos sintéticos

````{admonition} Resumen 
:class: tip

Este artículo presenta las líneas de trabajo desarrolladas para la generación de un dataset sintético de tamaño y peso de lenguados, con el objetivo de proporcionar una base de datos suficientemente amplia y representativa para el entrenamiento, validación y mejora de modelos predictivos. Se recogen las metodologías utilizadas en la generación de datos sintéticos, incluyendo la modelización estadística de distribuciones empíricas, técnicas de simulación basadas en procesos de crecimiento biológico y enfoques de aprendizaje automático para la síntesis de datos realistas. Además, se analizan los criterios de validación empleados para garantizar que los datos generados reflejen fielmente las tendencias y variabilidad observadas en poblaciones reales de lenguado (*Solea solea*), asegurando así su utilidad en el desarrollo de algoritmos precisos y generalizables para la predicción del peso a partir de variables alometricas.

**Entregable**: E2.1  
**Versión**: 1.0  
**Autor**: Javier Álvarez Osuna  
**Email**: javier.osuna@fishfarmfeeder.com  
**ORCID**: [0000-0001-7063-1279](https://orcid.org/0000-0001-7063-1279)  
**Licencia**: CC-BY-4.0  
**Código proyecto**: IG408M.2025.000.000072

```{figure} .././assets/FLATCLASS_logo_publicidad.png
:width: 100%
:align: center
```

````

## Introducción

En el ámbito de la acuicultura de precisión, la caracterización morfométrica de los peces y su relación con el peso corporal constituye un eje central para la optimización de procesos como la clasificación automática, el control de crecimiento y la dosificación alimentaria. En el caso particular de los peces planos en fase de alevinaje —como el lenguado (Solea solea) o el rodaballo (Scophthalmus maximus)—, las variables morfométricas fundamentales incluyen la longitud corporal, la anchura transversal y la altura dorso-ventral, parámetros que definen la geometría del individuo y que se presumen relacionados de forma sistemática con la biomasa individual.

La necesidad de disponer de un dataset suficientemente amplio, representativo y multivariado, que relacione estas variables morfométricas con el peso corporal correspondiente, responde a múltiples consideraciones de carácter estadístico, biológico y computacional. Aun en ausencia de un modelo alométrico explícito que relacione de forma determinista dichas variables, es posible anticipar que cualquier estrategia de inferencia o predicción del peso basada en dimensiones requerirá una densidad adecuada de datos en el espacio tridimensional definido por longitud, anchura y altura. Este requisito es crítico para garantizar tanto la fidelidad del ajuste como la capacidad de generalización del modelo aprendido.

Cuando el volumen de datos disponibles es reducido, surgen una serie de limitaciones estructurales:

- **Alta varianza en la estimación de parámetros**: La precisión de los modelos predictivos decae significativamente cuando las observaciones son escasas o están mal distribuidas en el dominio de entrada.
- **Riesgo de sobreajuste**: En entornos de datos reducidos, los modelos tienden a capturar ruido en lugar de relaciones funcionales genuinas, lo cual compromete la validez externa.
- **Cobertura insuficiente del espacio morfométrico**: Se produce una pérdida de representatividad en las regiones marginales del dominio, lo que reduce la capacidad del sistema para extrapolar o interpolar en condiciones reales de producción.
- **Sesgos estructurales**: Las muestras pequeñas pueden reflejar sesgos en las condiciones de cría, genética o instrumentación, induciendo patrones espurios no generalizables.

Desde la perspectiva de la matemática probabilística, esta problemática puede entenderse mediante el marco de la inferencia bayesiana. En este enfoque, el conocimiento sobre los parámetros $\theta$ (por ejemplo, la relación entre morfología y peso) se representa como una distribución posterior condicionada a los datos $D$:

$$
p(\theta | D) \propto p(D | \theta) \cdot p(\theta)
$$

donde $D$ representa los datos observados. Cuando el tamaño de $D$ es reducido, la función de verosimilitud $p(D|a,b)$ tiene una varianza alta, lo que genera estimaciones más inciertas. Al aumentar el tamaño de $D$ con datos sintéticos plausibles, la estimación de $p(a,b|D)$ se vuelve más precisa, reduciendo la varianza de los parámetros.

En este trabajo se abordan en profundidad las estrategias metodológicas implementadas para la construcción de un conjunto de datos sintético destinado a la estimación del peso en alevines de lenguado (*Solea solea*), a partir de variables morfométricas. El objetivo central consiste en disponer de un volumen de datos suficientemente amplio y diversificado que permita fortalecer las fases de entrenamiento, validación cruzada y ajuste fino del modelo predictivo alométrico. Para ello, se ha empleado una combinación de técnicas de inferencia estadística multivariante, simulación probabilística de estructuras corporales y generación asistida por modelos matemáticos del crecimiento. El estudio incluye una evaluación rigurosa de la coherencia morfobiológica de los datos generados, así como la verificación de que estos reproducen con fidelidad las correlaciones estructurales presentes en la muestra observacional original. Todo ello con el propósito de proporcionar una base fiable para el despliegue de algoritmos de predicción del peso que sean robustos, precisos y extrapolables a contextos de aplicación reales en acuicultura.